In [1]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import pandas as pd
import re

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import re 
import string 
import time
pd.set_option("display.max_rows",50)
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate

from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#df= pd.read_csv("cleaned_dataset.csv")

In [3]:
#df.head()

In [4]:
#df.shape

In [5]:
#def remove_folders_with_n_emails(emails,n):
#    email_count = dict(df['X-Folder'].value_counts())
#    small_folder = [key for key, value in email_count.items() if value<=n]
#    email=df.loc[~df['X-Folder'].isin(small_folder)]
#    return email

In [6]:
#n = 150
#df = remove_folders_with_n_emails(df, n)

In [7]:
#df.head(3)

In [8]:
#df.shape

In [9]:
#print("total unique folders :",len(df['X-Folder'].unique()))

In [10]:
#df['text'] = df['Subject'] + " " + df['body']

In [11]:
#df.head(3)

In [12]:
## remove subject and body column from the dataset
#drop_col = ['Subject','body']
#df.drop(columns=drop_col,axis=1, inplace=True)

In [13]:
#df.head(3)

In [14]:
#stopword = stopwords.words('english')

In [15]:
    # lowercase words
#def preprocess_text(text):
#    text = text.lower()
#    
#    # remove new line 
#    text = re.sub(r"\n+"," ",text)
#    
#    #remove tabs,punctuation and commas
#    text= re.sub("["+string.punctuation+"]", " ", text)
#    
#    # remove extra white space
#    text = re.sub("\s+"," ",text)
#    
#    # remove stopwords from text
#    text =" ".join([word for word in text.split() if word not in stopword])
#    return text
#
#start = time.time()
#df['text'] = df['text'].apply(lambda x : preprocess_text(x))

In [16]:
#start= time.time()
#folder_dict = dict(df['X-Folder'].value_counts().sort_values()[50:70])
#data = df[df['X-Folder'].isin(folder_dict.keys())]
#end= time.time()
#print("total time for execution  :",(end-start))

In [17]:
#data.to_csv("preprocessed.csv",index=False)

In [18]:
data = pd.read_csv("preprocessed.csv")
data.head()

,X-Folder,text
0,california,caiso notice summer 2001 generation rfb market...
1,california,ca iso cal px information related 2000 market ...
2,california,caiso notification update inter sc trades adju...
3,california,update mif meeting presentations iso website u...
4,california,mif presentations presentations market issues ...


In [19]:
categories = data['X-Folder'].unique()
categories

array(['california', 'calendar', 'resumes', 'archives', 'junk file',
       'ces', 'logistics', 'tufco', 'projects', 'online trading',
       'management', 'esvl', 'ooc', 'corporate', 'tw-commercial group',
       'genco-jv_ipo', 'junk', 'deal discrepancies', 'e-mail bin',
       'bill williams iii'], dtype=object)

In [20]:
def label_encoder(data):
    encoder = LabelEncoder()
    # apply encoder on dependent feature means X-Folder
    y = encoder.fit_transform(data['X-Folder'])
    return y

In [21]:
y = label_encoder(data)
input_data = data['text']

# Let's get a bag of words

In [22]:
start = time.time()
count_vect = CountVectorizer(min_df=5, max_features=5000)
X = count_vect.fit_transform(input_data)
end = time.time()
print("Execution time :",(end-start))

Execution time : 2.2493438720703125


In [23]:
start= time.time()
X= X.toarray()
print("X shape : ",X.shape)
end= time.time()
print("Execution time :",(end-start))

X shape :  (13586, 5000)
Execution time : 0.48056936264038086


In [24]:
# create dataframe to store results
f1_data = {
    'Algorithm': ['Gaussian NB', 'Multinomial NB','Decision Tree','SVM'],
    'BoW': ''
}


jaccard_data = {
    'Algorithm': ['Gaussian NB', 'Multinomial NB', 'Decision Tree','SVM'],
    'BoW': ''
}


acc_data = {
    'Algorithm': ['Gaussian NB', 'Multinomial NB','Decision Tree','SVM'],
    'BoW': ''
}

estimator_data = {
    'Algorithm': ['Gaussian NB', 'Multinomial NB','Decision Tree','SVM'],
    'BoW': ''
}

f1_df= pd.DataFrame(f1_data)
jac_df= pd.DataFrame(jaccard_data)
acc_df= pd.DataFrame(acc_data)
estimator_df= pd.DataFrame(estimator_data)

In [25]:
# Training and Evaluation
models =[GaussianNB(),MultinomialNB(), DecisionTreeClassifier(), LinearSVC()]

names=['GaussianNB','MultinomialNB','DecisionTreeClassifier','LinearSVC']


jacc_scores = []
acc_scores = []
f1_scores = []
exec_times = []
estimators = []

In [26]:
for model, name in zip(models,names):
    print(model)
    start= time.time()
    scoring ={"acc":'accuracy',
             'f1_mac':'f1_macro',
             'jacc_mac':'jaccard_macro'}
    scores = cross_validate(model,X,y,cv=5, n_jobs=4,scoring=scoring, return_estimator=True)
    training_time= (time.time()-start)
    print("accuracy: ", scores['test_acc'].mean())
    print("f1_score: ", scores['test_f1_mac'].mean())
    print("Jaccard_index: ", scores['test_jacc_mac'].mean())
    print("time (sec): ", training_time)
    print("\n")
    jacc_scores.append(scores['test_jacc_mac'].mean())
    acc_scores.append(scores['test_acc'].mean())
    f1_scores.append(scores['test_f1_mac'].mean())
    exec_times.append(training_time)
    print("Estimator: ", type(scores['estimator']), scores['estimator'])
    estimators.append(scores['estimator'])

GaussianNB()
accuracy:  0.5668335498589943
f1_score:  0.5441217915003902
Jaccard_index:  0.392009682672538
time (sec):  19.559530019760132


Estimator:  <class 'list'> [GaussianNB(), GaussianNB(), GaussianNB(), GaussianNB(), GaussianNB()]
MultinomialNB()
accuracy:  0.7278835760885255
f1_score:  0.6939782155546919
Jaccard_index:  0.5628269573042803
time (sec):  45.17524981498718


Estimator:  <class 'list'> [MultinomialNB(), MultinomialNB(), MultinomialNB(), MultinomialNB(), MultinomialNB()]
DecisionTreeClassifier()
accuracy:  0.6400716281510983
f1_score:  0.6224774546051386
Jaccard_index:  0.4720429285793058
time (sec):  89.39858531951904


Estimator:  <class 'list'> [DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier()]
LinearSVC()
accuracy:  0.7163261702473972
f1_score:  0.6985491164524109
Jaccard_index:  0.5614889087564536
time (sec):  40.389232873916626


Estimator:  <class 'list'> [LinearSVC(), LinearSVC()

In [27]:
acc_df['BoW'] = acc_scores
f1_df['BoW'] =f1_scores
jac_df['BoW'] =jacc_scores
acc_df['BoW_time'] = training_time
estimator_df['BoW'] = estimators


In [28]:
acc_df['BoW_time'] = training_time

In [29]:
acc_df

,Algorithm,BoW,BoW_time
0,Gaussian NB,0.566834,40.389233
1,Multinomial NB,0.727884,40.389233
2,Decision Tree,0.640072,40.389233
3,SVM,0.716326,40.389233


# Bag of Words Bigram

In [30]:
start = time.time()
count_vect = CountVectorizer(min_df=5, max_features=5000,ngram_range=(2,2))
X= count_vect.fit_transform(input_data)
X = X.toarray()
print("X shape :",X.shape)
end = time.time()
print("Execution time :",(end-start))

X shape : (13586, 5000)
Execution time : 6.158414840698242


In [31]:
models =[GaussianNB(),MultinomialNB(), DecisionTreeClassifier(), LinearSVC()]

names=['GaussianNB','MultinomialNB','DecisionTreeClassifier','LinearSVC']


jacc_scores = []
acc_scores = []
f1_scores = []
exec_times = []
estimators = []

for model, name in zip(models,names):
    print(model)
    start= time.time()
    scoring ={"acc":'accuracy',
             'f1_mac':'f1_macro',
             'jacc_mac':'jaccard_macro'}
    scores = cross_validate(model,X,y,cv=5, n_jobs=4,scoring=scoring, return_estimator=True)
    training_time= (time.time()-start)
    print("accuracy: ", scores['test_acc'].mean())
    print("f1_score: ", scores['test_f1_mac'].mean())
    print("Jaccard_index: ", scores['test_jacc_mac'].mean())
    print("time (sec): ", training_time)
    print("\n")
    jacc_scores.append(scores['test_jacc_mac'].mean())
    acc_scores.append(scores['test_acc'].mean())
    f1_scores.append(scores['test_f1_mac'].mean())
    exec_times.append(training_time)
    print("Estimator: ", type(scores['estimator']), scores['estimator'])
    estimators.append(scores['estimator'])
    
    
acc_df['BoWBi'] = acc_scores
f1_df['BoWBi'] =f1_scores
jac_df['BoWBi'] =jacc_scores
estimator_df['BoWBi'] = estimators
acc_df['BoWBi_time'] = training_time

acc_df

GaussianNB()
accuracy:  0.5741205388469244
f1_score:  0.5531704529630934
Jaccard_index:  0.39584816767646086
time (sec):  17.054582118988037


Estimator:  <class 'list'> [GaussianNB(), GaussianNB(), GaussianNB(), GaussianNB(), GaussianNB()]
MultinomialNB()
accuracy:  0.6235102452251284
f1_score:  0.6041764725559157
Jaccard_index:  0.45575511615901265
time (sec):  46.43130564689636


Estimator:  <class 'list'> [MultinomialNB(), MultinomialNB(), MultinomialNB(), MultinomialNB(), MultinomialNB()]
DecisionTreeClassifier()
accuracy:  0.5744900543088065
f1_score:  0.562630778689734
Jaccard_index:  0.41238226582720844
time (sec):  233.63268423080444


Estimator:  <class 'list'> [DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier()]
LinearSVC()
accuracy:  0.6187991397472052
f1_score:  0.6059513636599976
Jaccard_index:  0.45511254094764997
time (sec):  26.55026149749756


Estimator:  <class 'list'> [LinearSVC(), LinearS

,Algorithm,BoW,BoW_time,BoWBi,BoWBi_time
0,Gaussian NB,0.566834,40.389233,0.574121,26.550261
1,Multinomial NB,0.727884,40.389233,0.623510,26.550261
2,Decision Tree,0.640072,40.389233,0.574490,26.550261
3,SVM,0.716326,40.389233,0.618799,26.550261


# TFIDF

In [32]:
start= time.time()
tfidf= TfidfVectorizer(min_df=5,max_features=5000)
X = tfidf.fit_transform(input_data)
X= X.toarray()
print("X shape :",X.shape)
end = time.time()
print("execution time :",(end-start))

X shape : (13586, 5000)
execution time : 2.5676565170288086


In [33]:
models =[GaussianNB(),MultinomialNB(), DecisionTreeClassifier(), LinearSVC()]

names=['GaussianNB','MultinomialNB','DecisionTreeClassifier','LinearSVC']


jacc_scores = []
acc_scores = []
f1_scores = []
exec_times = []
estimators = []

for model, name in zip(models,names):
    print(model)
    start= time.time()
    scoring ={"acc":'accuracy',
             'f1_mac':'f1_macro',
             'jacc_mac':'jaccard_macro'}
    scores = cross_validate(model,X,y,cv=5, n_jobs=4,scoring=scoring, return_estimator=True)
    training_time= (time.time()-start)
    print("accuracy: ", scores['test_acc'].mean())
    print("f1_score: ", scores['test_f1_mac'].mean())
    print("Jaccard_index: ", scores['test_jacc_mac'].mean())
    print("time (sec): ", training_time)
    print("\n")
    jacc_scores.append(scores['test_jacc_mac'].mean())
    acc_scores.append(scores['test_acc'].mean())
    f1_scores.append(scores['test_f1_mac'].mean())
    exec_times.append(training_time)
    print("Estimator: ", type(scores['estimator']), scores['estimator'])
    estimators.append(scores['estimator'])
    
acc_df['tfidf'] = acc_scores
f1_df['tfidf'] =f1_scores
jac_df['tfidf'] =jacc_scores
estimator_df['tfidf'] = estimators
acc_df['tfidf_timing'] = training_time
acc_df

GaussianNB()
accuracy:  0.5836893751846698
f1_score:  0.5630253882669244
Jaccard_index:  0.4115797560251563
time (sec):  18.5907084941864


Estimator:  <class 'list'> [GaussianNB(), GaussianNB(), GaussianNB(), GaussianNB(), GaussianNB()]
MultinomialNB()
accuracy:  0.72302468067543
f1_score:  0.6815725509632343
Jaccard_index:  0.5508293556366095
time (sec):  3.643803834915161


Estimator:  <class 'list'> [MultinomialNB(), MultinomialNB(), MultinomialNB(), MultinomialNB(), MultinomialNB()]
DecisionTreeClassifier()
accuracy:  0.6374959883847999
f1_score:  0.6209855262382802
Jaccard_index:  0.4697991480621832
time (sec):  107.30777287483215


Estimator:  <class 'list'> [DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier(), DecisionTreeClassifier()]
LinearSVC()
accuracy:  0.7836020066065379
f1_score:  0.7668982829267701
Jaccard_index:  0.6454489354776655
time (sec):  8.871842861175537


Estimator:  <class 'list'> [LinearSVC(), LinearSVC(), L

,Algorithm,BoW,BoW_time,BoWBi,BoWBi_time,tdidf,tdidf_timing
0,Gaussian NB,0.566834,40.389233,0.574121,26.550261,0.583689,8.871843
1,Multinomial NB,0.727884,40.389233,0.623510,26.550261,0.723025,8.871843
2,Decision Tree,0.640072,40.389233,0.574490,26.550261,0.637496,8.871843
3,SVM,0.716326,40.389233,0.618799,26.550261,0.783602,8.871843


# Bag of Words Multinomial NB performed best, lets fit that model

In [61]:
start= time.time()
tfidf= TfidfVectorizer(min_df=5,max_features=5000)
X = tfidf.fit_transform(input_data)
X= X.toarray()
print("X shape :",X.shape)
end = time.time()
print("execution time :",(end-start))

X shape : (13586, 5000)
execution time : 3.406904458999634


In [35]:
#start= time.time()
#X= X.toarray()
#print("X shape : ",X.shape)
#end= time.time()
#print("Execution time :",(end-start))

X shape :  (13586, 5000)
Execution time : 0.5003066062927246


In [63]:
models[3].fit(X, y)

LinearSVC()

# Set up IMAP to interact with Outlook account

In [64]:
#input = []
#input.append('California california los angeles orange county')
#X = count_vect.transform(input)
#X = X.toarray()
##print(type(models[1].predict(X)))
#pred_category = categories[models[1].predict(X)[0]]

In [65]:
# account credentials
username = "Your_email_address"
password = "Your_password"
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for office 365, it's this:
imap_server = "imap-mail.outlook.com"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [66]:
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)

('OK', [b'LOGIN completed.'])

In [67]:
status, messages = imap.select("INBOX")
# number of top emails to fetch
#N = int(messages[0])
N = int(messages[0])
# total number of emails
messages = int(messages[0])

# List containing subject, from, and body
emailsList = []

In [68]:
def parse_uid(msg_data):
    #print(type(msg_data))
    pattern_uid = re.compile(r'\d+ \(UID (?P<uid>\d+)\)')
    match = pattern_uid.match(msg_data)
    return match.group('uid')

In [69]:
def get_uid_and_rfc822(folder):
    imap.select(folder)
    raw_emails = []

    result, data = imap.uid('search', None, "ALL")
    ids = data[0]
    id_list = ids.split()

    for i in id_list:
        res, msg = imap.uid('fetch', i, '(RFC822)')
        #raw_email = msg[0][1]
        #raw_emails.append(raw_email)
        for response in msg:
                #rfc_string = response[0].decode()
                #print(type(rfc_string))
                if isinstance(response, tuple):

                    # parse a bytes email into a message object
                    msg = email.message_from_bytes(response[1])
                    # decode the email subject
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        # if it's a bytes, decode to str
                        subject = subject.decode(encoding)
                    # decode email sender
                    From, encoding = decode_header(msg.get("From"))[0]
                    if isinstance(From, bytes):
                        From = From.decode(encoding)
                    print("Subject:", subject)
                    print("From:", From)
                    # if the email message is multipart
                    if msg.is_multipart():
                        # iterate over email parts
                        for part in msg.walk():
                            # extract content type of email
                            content_type = part.get_content_type()
                            content_disposition = str(part.get("Content-Disposition"))
                            try:
                                # get the email body
                                body = part.get_payload(decode=True).decode()
                            except:
                                pass
                            if content_type == "text/plain" and "attachment" not in content_disposition:
                                # print text/plain emails and skip attachments
                                #print("This is the plain body: " + body)

                                #print("Whats happening to the body??: " + body)
                                emailGroup = [i, subject, From, body]
                                print('EMAIL GROUP', emailGroup)
                                print(type(emailsList))
                                emailsList.append(emailGroup)
                            elif "attachment" in content_disposition:
                                print('sad')
                                # download attachment
                                filename = part.get_filename()
                                if filename:
                                    folder_name = clean(subject)
                                    if not os.path.isdir(folder_name):
                                        # make a folder for this email (named after the subject)
                                        os.mkdir(folder_name)
                                    filepath = os.path.join(folder_name, filename)
                                    # download attachment and save it
                                    open(filepath, "wb").write(part.get_payload(decode=True))
                    else:
                        # extract content type of email
                        content_type = msg.get_content_type()
                        # get the email body
                        body = msg.get_payload(decode=True).decode()
                        if content_type == "text/plain":
                            # print only text email parts
                            print(body)
                    print("="*100)

            #return id_list, raw_emails
            

In [70]:
get_uid_and_rfc822('Inbox')

Subject: Bill Williams III
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'85', 'Bill Williams III', 'Stephen L <stephen.lanna1994@gmail.com>', 'Invitation to attend a talk by Bill Williams III\r\n']
<class 'list'>
Subject: Online Trading
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'106', 'Online Trading', 'Stephen L <stephen.lanna1994@gmail.com>', 'Invitation to an online trading seminar\r\n']
<class 'list'>
Subject: Archives
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'129', 'Archives', 'Stephen L <stephen.lanna1994@gmail.com>', 'Request for access to archived material\r\n']
<class 'list'>
Subject: Resumes
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'130', 'Resumes', 'Stephen L <stephen.lanna1994@gmail.com>', 'Request for review of resumes\r\n']
<class 'list'>
Subject: Junk
From: Stephen L <stephen.lanna1994@gmail.com>
EMAIL GROUP [b'131', 'Junk', 'Stephen L <stephen.lanna1994@gmail.com>', 'Warning about a potential junk em

In [71]:
emailDf = pd.DataFrame(emailsList, columns=['UID', 'Subject', 'From', 'Body'])

In [72]:
emailDf.head()

,UID,Subject,From,Body
0,b'85',Bill Williams III,Stephen L <stephen.lanna1994@gmail.com>,Invitation to attend a talk by Bill Williams I...
1,b'106',Online Trading,Stephen L <stephen.lanna1994@gmail.com>,Invitation to an online trading seminar\r\n
2,b'129',Archives,Stephen L <stephen.lanna1994@gmail.com>,Request for access to archived material\r\n
3,b'130',Resumes,Stephen L <stephen.lanna1994@gmail.com>,Request for review of resumes\r\n
4,b'131',Junk,Stephen L <stephen.lanna1994@gmail.com>,Warning about a potential junk email\r\n


In [73]:
def combine_columns(emaildf):
    emaildf['Text'] = emaildf['Subject'] + ' ' + emaildf['Body']
    emaildf = emaildf.drop('Subject', axis=1)
    emaildf = emaildf.drop('Body', axis=1)
    return emaildf

In [74]:
emailDf = combine_columns(emailDf)
emailDf.head()

,UID,From,Text
0,b'85',Stephen L <stephen.lanna1994@gmail.com>,Bill Williams III Invitation to attend a talk ...
1,b'106',Stephen L <stephen.lanna1994@gmail.com>,Online Trading Invitation to an online trading...
2,b'129',Stephen L <stephen.lanna1994@gmail.com>,Archives Request for access to archived materi...
3,b'130',Stephen L <stephen.lanna1994@gmail.com>,Resumes Request for review of resumes\r\n
4,b'131',Stephen L <stephen.lanna1994@gmail.com>,Junk Warning about a potential junk email\r\n


# Create necessary folders

In [75]:
for cat in categories:
  cat = cat.title().replace(" ", "")

In [76]:
print(categories)

['california' 'calendar' 'resumes' 'archives' 'junk file' 'ces'
 'logistics' 'tufco' 'projects' 'online trading' 'management' 'esvl' 'ooc'
 'corporate' 'tw-commercial group' 'genco-jv_ipo' 'junk'
 'deal discrepancies' 'e-mail bin' 'bill williams iii']


# Classify emails

In [77]:
print(estimator_df['BoW'])

0    [GaussianNB(), GaussianNB(), GaussianNB(), Gau...
1    [MultinomialNB(), MultinomialNB(), Multinomial...
2    [DecisionTreeClassifier(), DecisionTreeClassif...
3    [LinearSVC(), LinearSVC(), LinearSVC(), Linear...
Name: BoW, dtype: object


In [78]:
#models[1].predict(X)

In [79]:
#input = []
#input.append('California california los angeles orange county')
#X = count_vect.transform(input)
#X = X.toarray()
#pred_category = categories[estimator_df['BoW'][1][4].predict(X)[0]]

In [80]:
# Loop through emailDf and predict category using best performing estimator
pred_dict = {}
for index, row in emailDf.iterrows():
    input = []
    input.append(row['Text'])
    X = count_vect.transform(input)
    X = X.toarray()
    pred_category = categories[models[3].predict(X)[0]]
    pred_dict[row['UID']] = pred_category

In [81]:
# Function to sort emails. Takes a dictionary with a UID as key,
# predicted category as value
def sort_by_category(pred_dict):
  for uid in pred_dict.keys():
    #print(categories[pred_dict.get(uid)])
    cat = pred_dict.get(uid).title().replace(" ", "")
    print(cat)
    result = imap.uid('COPY', uid, cat)

    if result[0] == 'OK':
      mov, data = imap.uid('STORE', uid , '+FLAGS', '(\Deleted)')
      imap.expunge()

In [82]:
sort_by_category(pred_dict)

Calendar
Tw-CommercialGroup
BillWilliamsIii
DealDiscrepancies
Resumes
Resumes
Ooc
Resumes
Ooc
BillWilliamsIii
Archives
Tw-CommercialGroup
Genco-Jv_Ipo
Archives
Ooc
Ooc
Logistics
JunkFile
JunkFile
E-MailBin
DealDiscrepancies
DealDiscrepancies
Corporate
Archives
Archives


In [83]:
input = []
input.append("California")

In [84]:
X = count_vect.transform(input)
X = X.toarray()

In [85]:
X.shape

(1, 5000)

In [86]:
pred_category = categories[estimator_df['BoW'][1][0].predict(X)[0]]
pred_category

'archives'